First Goal is to make the computer understand the language<br>

Here are the steps

1. Sentence segmentation
2. Word tokenization
3. Stemming
4. Lemmatization
5. Stop word analysis
6. Dependency parsing
7. Part-of-speech (POS) tagging

In [136]:
import numpy as np
import re

**Prepare a proper storage**

In [124]:
class Tweet:
    def __init__(self, count, hate_speech_count, offensive_language_count, neither_count, classs, tweet):
        """
        - count (int): The total count of the tweet.
        - hate (int): The count of hate speech in the tweet.
        - offensive (int): The count of offensive language in the tweet.
        - neither (int): The count of content classified as neither hate speech nor offensive.
        - classs (str): The classification of the tweet.
        - tweet (str): The text content of the tweet.
        """
        self.count = count
        self.hate = hate_speech_count
        self.offensive = offensive_language_count
        self.neither = neither_count
        self.classs = classs
        self.tweet = tweet

    def __str__(self):
        return f"{self.count} ; {self.hate} ; {self.offensive} ; {self.neither} ; {self.classs} ;; {self.tweet}"

**Load the data**

In [125]:
Tweets = []

In [130]:
with open('./archive/train.csv', 'r+') as file:
    previous_line = ''

    # Initialize a list to accumulate the modified content
    final_content_lines = []

    # Read and accumulate non-empty lines
    for line in file:
        stripped_line = line.strip()

        if stripped_line and stripped_line[0].isdigit():
            # If the current line is not empty and starts with an integer, accumulate it
            final_content_lines.append(stripped_line)
            previous_line = stripped_line
        else:
            # If the current line doesn't start with an integer, append it to the previous line
            previous_line += stripped_line

In [131]:
for line in final_content_lines:
    
    comma_indices = [index for index, char in enumerate(line) if char == ',']

    # Extracting substrings between commas
    count_str = line[0:comma_indices[0]].strip()
    hate_str = line[comma_indices[0]+1:comma_indices[1]].strip()
    offensive_str = line[comma_indices[1]+1:comma_indices[2]].strip()
    neither_str = line[comma_indices[2]+1:comma_indices[3]].strip()
    classs_str = line[comma_indices[3]+1:comma_indices[4]].strip()
    tweet_str = line[comma_indices[4]+1:].strip()

    # Converting to integers
    count = int(count_str) if count_str.isdigit() else None
    hate = int(hate_str) if hate_str.isdigit() else None
    offensive = int(offensive_str) if offensive_str.isdigit() else None
    neither = int(neither_str) if neither_str.isdigit() else None
    classs = int(classs_str) if classs_str.isdigit() else None

    # Creating an instance of the Tweet class
    tweet_instance = Tweet(count, hate, offensive, neither, classs_str, tweet_str)

    # Append the tweet instance to a list or do whatever you need to do with it
    Tweets.append(tweet_instance)

In [ ]:
for t in Tweets[:10]:
    print(t)

**Tweet formatting**

In [146]:
for t in Tweets:
    t.tweet = t.tweet.replace('RT', '').replace('!', '').replace('"', '').replace("\n", '')
    for _ in range(5):
        t.tweet = t.tweet.replace('  ', ' ').replace('..', '.') # remove multiple points & space
    
for t in Tweets[:10]:
    print(t)

3 ; 0 ; 0 ; 3 ; 2 ;; @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out.
3 ; 0 ; 3 ; 0 ; 1 ;; @mleew17: boy dats cold.tyga dwn bad for cuffin dat hoe in the 1st place
3 ; 0 ; 3 ; 0 ; 1 ;; @UrKindOfBrand Dawg @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3 ; 0 ; 2 ; 1 ; 1 ;; @C_G_Anderson: @viva_based she look like a tranny
6 ; 0 ; 6 ; 0 ; 1 ;; @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
3 ; 1 ; 2 ; 0 ; 1 ;; @T_Madison_x: The shit just blows me.claim you so faithful and down for somebody but still fucking with hoes &#128514;&#128514;&#128514;
3 ; 0 ; 3 ; 0 ; 1 ;; @__BrighterDays: I can not just sit up and HATE on another bitch . I got too much shit going on
3 ; 0 ; 3 ; 0 ; 1 ;; &#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls&#8221;
3 ; 0 ; 3 ; 0 ; 1 ;; &amp; 

As **emojis** also have their meaning, i will extract them

In [164]:
emojis = {}
for t in Tweets:
    emoji_indices = [match.start() for match in re.finditer(r'&#', t.tweet)]
    for em in emoji_indices:
        em += 2 # skip &#
        end = em
        while t.tweet[end].isdigit(): # go until the last digit
            end += 1
        emojis[t.tweet[em:end]] = None

In [169]:
print((set(emojis.keys())))

{'1581', '127867', '128700', '127817', '127827', '10060', '128066', '127481', '9898', '201', '127856', '128203', '128282', '127936', '8419', '128096', '128533', '1041233', '1043473', '58380', '127468', '127861', '128008', '65533', '127984', '9918', '127942', '127809', '128122', '187', '127770', '128025', '128588', '128581', '128179', '9749', '1043214', '127812', '9917', '128167', '8482', '1043350', '128133', '1590', '1041219', '128209', '128572', '127913', '160', '128270', '127837', '128549', '128174', '1041237', '57423', '127830', '9924', '128548', '1040788', '10024', '57933', '128054', '127805', '247', '776', '58399', '128094', '1041196', '176', '128683', '58407', '1043215', '9855', '1594', '127866', '65039', '1041205', '127880', '127382', '9995', '128077', '128524', '1043323', '128013', '8242', '128563', '128517', '128661', '128558', '128550', '127912', '128221', '128072', '128138', '127882', '128117', '128155', '8986', '1041204', '128058', '128665', '127379', '128518', '128184', '1

Input the keys in Chat-GPT and ask it to generate some **description** of the emoji

In [ ]:
emoji_mapping = {
    57361: [57361],
    128514: [128514, 128517, 128591, 128557, 128531, 128588, 128515, 128512, 128513, 128514, 128515, 128516, 128517, 128518, 128519, 128520, 128521, 128522, 128523, 128524, 128525, 128526, 128527, 128528, 128529, 128530, 128531, 128532, 128533, 128534, 128535, 128536, 128537, 128538, 128539, 128540, 128541, 128542, 128543, 128544, 128545, 128546, 128547, 128548, 128549, 128550, 128551, 128552, 128553, 128554, 128555, 128556, 128557, 128558, 128559, 128560, 128561, 128562, 128563, 128564, 128565, 128566, 128567, 128568, 128569, 128570, 128571, 128572, 128573, 128574, 128575, 128576, 128577, 128578, 128579, 128580, 128581, 128582, 128583, 128584, 128585, 128586, 128587, 128588, 128589, 128590, 128591],
    128175: [128175, 128079],
    128553: [128553],
    128584: [128584],
    128175: [128175],
    128049: [128049],
    128554: [128554],
    128527: [128527],
    128056: [128056],
    128544: [128544],
    128545: [128545],
    128514: [128514]
}